# Préparation des données

- trouver/supprimer les données dupliquées : df.duplicated() / df.drop_duplicates()
- renommer des colonnes : df.rename(columns={...))
- trouver les NaN : df.isnull() / df.notnull() / df.dropna()
- travail sur les chaines : series.str.extract(), series.str.contains(), series.get_dummies()
- mapping : series.map()
- changer le type d'une série (cast) : df.astype(type) / pd.to_numeric() / pd.to_datetime()
- remplacer n'importe quelle valeur : df.replace({...})
- remplacer les NaN : df.fillna(), series.combine_first()

In [16]:
# imports
import numpy as np
import pandas as pd

In [4]:
%pycat people.csv

#### Chargement et analyse des données

In [6]:
pd.read_csv?

In [39]:
# load data
df = pd.read_csv('people.csv')
df

,id,first_name,last_name,email address,gender,age,money,lon,lat,phone,registration,inactive,last_seen,address,preference
0,27625,Leandra,Pabelik,lpabelik5f@yale.edu,Female,63,$55.18,18.284100,49.632552,0136319724,2019/04/16,False,1.559566e+09,"Palkovice, Czech Republic",entrée/plat/dessert
1,27570,Ruthi,Ross,rross3w@sohu.com,Female,57,$20.37,19.230220,50.466575,NaN,2018/10/23,False,1.567165e+09,"Siewierz, Poland",entrée/plat/dessert
2,27572,Silas,Stourton,silas.stourton3y@answers.com,Male,22,"€32,99",118.831081,24.984813,NaN,2018/12/30,False,1.553692e+09,"Longbo, China",entrée/plat/dessert
3,27435,Roxine,Pettecrew,rpettecrew5@gravatar.com,F,64,"£98,93",121.648987,-8.844744,NaN,2019/03/12,False,1.552349e+09,"Potulando, Indonesia",entrée/plat
4,27558,Margaux,Gowanson,nope@thankyou.,Female,54,$13.30,14.772557,45.160472,NaN,2018/08/13,False,1.543383e+09,"Bribir, Croatia",entrée/plat/dessert
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
204,27465,Adelind,Christin,achristinz@blogs.com,Female,30,"€81,84",41.427853,52.415968,NaN,2019/06/04,False,1.559606e+09,"Znamenka, Russia",entrée/plat/dessert
205,27476,Petronella,Pickance,ppickance1a@uiuc.edu,Female,28,€37.15,132.702111,33.762296,NaN,2017/11/22,False,1.532870e+09,"Iyo, Japan",entrée/plat/dessert
206,27521,Moritz,Issacof,missacof2j@wired.com,Male,21,$91.97,72.311099,31.263396,0298949280,2018/08/20,False,1.549817e+09,"Jhang Sadr, Pakistan",entrée/plat/dessert
207,27441,Quintana,Foulstone,qfoulstoneb@newsvine.com,F,32,"€55,64",111.138442,-6.783653,0734595126,2018/10/25,False,1.540426e+09,"Mantingantengah, Indonesia",entrée/plat/dessert


In [7]:
# info
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 209 entries, 0 to 208
Data columns (total 15 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             209 non-null    int64  
 1   first_name     207 non-null    object 
 2   last_name      207 non-null    object 
 3   email address  203 non-null    object 
 4   gender         207 non-null    object 
 5   age            207 non-null    object 
 6   money          190 non-null    object 
 7   lon            207 non-null    float64
 8   lat            207 non-null    float64
 9   phone          83 non-null     object 
 10  registration   207 non-null    object 
 11  inactive       207 non-null    object 
 12  last_seen      190 non-null    float64
 13  address        207 non-null    object 
 14  preference     207 non-null    object 
dtypes: float64(3), int64(1), object(11)
memory usage: 24.6+ KB


#### Duplications

- duplicated() : True ou False selon si une ligne est dupliquée
- drop_duplicates() : suppression des lignes dupliquées

In [8]:
# lignes dupliquées
df.duplicated().value_counts()

False    205
True       4
dtype: int64

In [9]:
# toutes les lignes dupliquées
df.loc[df.duplicated(keep=False)].sort_values('id')

,id,first_name,last_name,email address,gender,age,money,lon,lat,phone,registration,inactive,last_seen,address,preference
51,27445,Gardener,Brosi,gbrosif@flickr.com,M,47,€35.62,-39.071075,-11.256817,0321573819,2017/03/30,False,1.557537e+09,"Araci, Brazil",entrée/plat/dessert
148,27445,Gardener,Brosi,gbrosif@flickr.com,M,47,€35.62,-39.071075,-11.256817,0321573819,2017/03/30,False,1.557537e+09,"Araci, Brazil",entrée/plat/dessert
140,27472,Kennedy,Matzl,kmatzl16@prweb.com,Male,24,€66.59,118.679751,27.445870,NaN,2017/03/14,False,1.527821e+09,"Xinpu, China",entrée/plat/dessert
202,27472,Kennedy,Matzl,kmatzl16@prweb.com,Male,24,€66.59,118.679751,27.445870,NaN,2017/03/14,False,1.527821e+09,"Xinpu, China",entrée/plat/dessert
5,27497,Gordie,Bodicum,gbodicum1v@apache.org,Male,60,$47.26,13.978681,49.786243,0146943857,2017/04/29,False,1.545810e+09,"Jince, Czech Republic",entrée/plat/dessert
79,27497,Gordie,Bodicum,gbodicum1v@apache.org,Male,60,$47.26,13.978681,49.786243,0146943857,2017/04/29,False,1.545810e+09,"Jince, Czech Republic",entrée/plat/dessert
57,27547,Vasily,Moggach,vmoggach39@g.co,Male,39,"€47,01",20.254048,50.748099,0133026726,2018/01/07,False,1.538875e+09,"Złotniki, Poland",entrée/plat/dessert
93,27547,Vasily,Moggach,vmoggach39@g.co,Male,39,"€47,01",20.254048,50.748099,0133026726,2018/01/07,False,1.538875e+09,"Złotniki, Poland",entrée/plat/dessert


In [18]:
# suppression des lignes dupliquées
df = df.drop_duplicates()
len(df)

205

#### Renommage de la colonne 'email address'

In [43]:
# renommer les colonnes
df = df.rename(columns={'email address': 'email'})
df.head()

,id,first_name,last_name,email,gender,age,money,lon,lat,phone,registration,inactive,last_seen,address,preference,currency,money_eur
0,27625,Leandra,Pabelik,lpabelik5f@yale.edu,Female,63,$55.18,18.284100,49.632552,0136319724,2019/04/16,False,1.559566e+09,"Palkovice, Czech Republic",entrée/plat/dessert,USD,46.997700
1,27570,Ruthi,Ross,rross3w@sohu.com,Female,57,$20.37,19.230220,50.466575,NaN,2018/10/23,False,1.567165e+09,"Siewierz, Poland",entrée/plat/dessert,USD,17.349459
2,27572,Silas,Stourton,silas.stourton3y@answers.com,Male,22,"€32,99",118.831081,24.984813,NaN,2018/12/30,False,1.553692e+09,"Longbo, China",entrée/plat/dessert,EUR,32.990000
3,27435,Roxine,Pettecrew,rpettecrew5@gravatar.com,F,64,"£98,93",121.648987,-8.844744,NaN,2019/03/12,False,1.552349e+09,"Potulando, Indonesia",entrée/plat,GBP,108.815927
4,27558,Margaux,Gowanson,nope@thankyou.,Female,54,$13.30,14.772557,45.160472,NaN,2018/08/13,False,1.543383e+09,"Bribir, Croatia",entrée/plat/dessert,USD,11.327826


#### Analyse des données manquantes

numpy.nan est utilisé dans pandas pour représenter des valeurs manquantes.

In [12]:
# Not A Number
np.nan

nan

In [13]:
# type
type(np.nan)

float

In [14]:
# élément super absorbant
np.nan + 1

nan

In [15]:
# élément super absorbant
np.sqrt(np.nan)

nan

In [16]:
# élément super absorbant
np.nan == np.nan

False

In [17]:
# au passage infinis numpy
np.NINF, np.inf

(-inf, inf)

In [20]:
np.nan > 1

False

In [21]:
np.nan <= 1

False

In [19]:
np.inf > 1e100

True

In [22]:
np.inf + 1e100

inf

In [23]:
np.inf == np.inf

True

In [24]:
np.inf + np.NINF

nan

In [26]:
np.inf + np.inf > np.inf

False

#### Tests sur les données manquantes

- isnull() ou isna()
- notnull() ou notna()

In [29]:
df.loc[df['first_name']==np.nan]

,id,first_name,last_name,email,gender,age,money,lon,lat,phone,registration,inactive,last_seen,address,preference


In [37]:
# chercher les first_name Nan
df.loc[df['first_name'].isna()]

,id,first_name,last_name,email,gender,age,money,lon,lat,phone,registration,inactive,last_seen,address,preference
96,27565,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
200,27630,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [38]:
df.isnull()

,id,first_name,last_name,email,gender,age,money,lon,lat,phone,registration,inactive,last_seen,address,preference
0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
204,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False
205,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False
206,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
207,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False


In [39]:
# chercher tous les lignes avec au moins un NaN
df.loc[df.isnull().any(axis=1)]

,id,first_name,last_name,email,gender,age,money,lon,lat,phone,registration,inactive,last_seen,address,preference
1,27570,Ruthi,Ross,rross3w@sohu.com,Female,57,$20.37,19.230220,50.466575,NaN,2018/10/23,False,1.567165e+09,"Siewierz, Poland",entrée/plat/dessert
2,27572,Silas,Stourton,silas.stourton3y@answers.com,Male,22,"€32,99",118.831081,24.984813,NaN,2018/12/30,False,1.553692e+09,"Longbo, China",entrée/plat/dessert
3,27435,Roxine,Pettecrew,rpettecrew5@gravatar.com,F,64,"€98,93",121.648987,-8.844744,NaN,2019/03/12,False,1.552349e+09,"Potulando, Indonesia",entrée/plat
4,27558,Margaux,Gowanson,nope@thankyou.,Female,54,$13.30,14.772557,45.160472,NaN,2018/08/13,False,1.543383e+09,"Bribir, Croatia",entrée/plat/dessert
6,27539,Dulcine,Stopforth,dstopforth31@forbes.com,Female,22,€12.60,-35.587670,-6.255393,NaN,2017/12/13,False,1.547567e+09,"Serrinha, Brazil",entrée/plat/dessert
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200,27630,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
201,27519,Leonora,Guild,lguild2h@census.gov,Female,54,€54.57,122.195048,7.552335,NaN,2018/12/17,False,1.545005e+09,"Siraway, Philippines",entrée/plat/dessert
203,27612,Torrance,Sivills,tsivills52@mapquest.com,Male,24,$36.43,-91.148259,15.027043,NaN,2018/04/03,False,1.549641e+09,"Santa Cruz del Quiché, Guatemala",entrée/plat/dessert
204,27465,Adelind,Christin,achristinz@blogs.com,Female,30,"€81,84",41.427853,52.415968,NaN,2019/06/04,False,1.559606e+09,"Znamenka, Russia",entrée/plat/dessert


In [40]:
# suppression de toutes les lignes avec un NaN
df.dropna()

,id,first_name,last_name,email,gender,age,money,lon,lat,phone,registration,inactive,last_seen,address,preference
0,27625,Leandra,Pabelik,lpabelik5f@yale.edu,Female,63,$55.18,18.284100,49.632552,0136319724,2019/04/16,False,1.559566e+09,"Palkovice, Czech Republic",entrée/plat/dessert
5,27497,Gordie,Bodicum,gbodicum1v@apache.org,Male,60,$47.26,13.978681,49.786243,0146943857,2017/04/29,False,1.545810e+09,"Jince, Czech Republic",entrée/plat/dessert
15,27619,Shelley,Skeemor,sskeemor59@princeton.edu,Female,44,$41.50,111.878471,-7.584917,0547556034,2019/01/24,False,1.556976e+09,"Guyangan, Indonesia",entrée/plat/dessert
20,27462,Graig,Brownsea,gbrownseaw@arstechnica.com,M,54,$80.23,110.393439,-7.213328,0262855559,2018/05/22,False,1.569677e+09,"Sidomukti, Indonesia",entrée/plat/dessert
21,27466,Baron,Spurrett,bspurrett10@psu.edu,Male,36,$57.56,16.580506,-18.360404,06etcetera,2018/07/20,False,1.549132e+09,"Omuthiya, Namibia",entrée/plat/dessert
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197,27607,Mel,Pelz,mpelz4x@cbslocal.com,Male,57,€22.55,28.678979,7.267512,01.75.20.82.23,2018/10/16,False,1.563696e+09,"Tonj, South Sudan",entrée/plat/dessert
198,27624,Dur,Hallaways,dhallaways5e@delicious.com,Male,25,"€69,16",106.720976,28.739865,0317009357,2019/07/27,False,1.569215e+09,"Ganshui, China",entrée/plat/dessert
206,27521,Moritz,Issacof,missacof2j@wired.com,Male,21,$91.97,72.311099,31.263396,0298949280,2018/08/20,False,1.549817e+09,"Jhang Sadr, Pakistan",entrée/plat/dessert
207,27441,Quintana,Foulstone,qfoulstoneb@newsvine.com,F,32,"€55,64",111.138442,-6.783653,0734595126,2018/10/25,False,1.540426e+09,"Mantingantengah, Indonesia",entrée/plat/dessert


In [46]:
df.dropna().isnull().any(axis=1)

0      False
5      False
15     False
20     False
21     False
       ...  
197    False
198    False
206    False
207    False
208    False
Length: 71, dtype: bool

In [45]:
# suppression de toutes les lignes avec un NaN
df.dropna().isnull().any().any()

False

In [44]:
# supprimer uniquement les lignes dont le first_name NaN 
df = df.dropna(subset=['first_name'])
len(df)

207

#### Ajout d'une colonne 'full_name'

In [21]:
# 'full_name'  = 'first_name last_name'
df['full_name'] = df['first_name'] + ' ' + df['last_name']

#### Analyse de la colonne 'address'

In [48]:
# analyse de address
df['address'].value_counts()

Turka, Ukraine             3
Nova Odesa, Ukraine        2
Juntas, Costa Rica         1
Sunzhen, China             1
Sukajadi, Indonesia        1
                          ..
Dongshe, China             1
Cimrutu, Indonesia         1
Žlutava, Czech Republic    1
Araçatuba, Brazil          1
Stepanivka, Ukraine        1
Name: address, Length: 200, dtype: int64

#### Ajout des colonnes 'city' et 'country'

In [22]:
# calcul de city et country à partir de address
df[['city', 'country']] = df['address'].str.extract('(.*), (.*)')
df.head()

,id,first_name,last_name,email,gender,age,money,lon,lat,phone,registration,inactive,last_seen,address,preference,full_name,city,country
0,27625,Leandra,Pabelik,lpabelik5f@yale.edu,Female,63,$55.18,18.284100,49.632552,0136319724,2019/04/16,False,1.559566e+09,"Palkovice, Czech Republic",entrée/plat/dessert,Leandra Pabelik,Palkovice,Czech Republic
1,27570,Ruthi,Ross,rross3w@sohu.com,Female,57,$20.37,19.230220,50.466575,NaN,2018/10/23,False,1.567165e+09,"Siewierz, Poland",entrée/plat/dessert,Ruthi Ross,Siewierz,Poland
2,27572,Silas,Stourton,silas.stourton3y@answers.com,Male,22,"€32,99",118.831081,24.984813,NaN,2018/12/30,False,1.553692e+09,"Longbo, China",entrée/plat/dessert,Silas Stourton,Longbo,China
3,27435,Roxine,Pettecrew,rpettecrew5@gravatar.com,F,64,"€98,93",121.648987,-8.844744,NaN,2019/03/12,False,1.552349e+09,"Potulando, Indonesia",entrée/plat,Roxine Pettecrew,Potulando,Indonesia
4,27558,Margaux,Gowanson,nope@thankyou.,Female,54,$13.30,14.772557,45.160472,NaN,2018/08/13,False,1.543383e+09,"Bribir, Croatia",entrée/plat/dessert,Margaux Gowanson,Bribir,Croatia


#### Analyse des données

In [50]:
# nunique : modalités par colonne
df.nunique()

id              201
first_name      195
last_name       201
email           197
gender            4
age              54
money           184
lon             199
lat             199
phone            80
registration    183
inactive          1
last_seen       184
address         200
preference        5
full_name       201
city            200
country          60
dtype: int64

#### Mapping du genre

In [75]:
# analyse du gender
df['gender'].unique()

array(['Female', 'Male', 'F', 'M'], dtype=object)

In [8]:
# analyse du gender
df['gender'].value_counts()

Male      89
Female    79
M         22
F         13
Name: gender, dtype: int64

In [9]:
# traitement du gender
mapping = {'Female': 'F', 'Male': 'M', 'F': 'F', 'M': 'M'}
df['gender'] = df['gender'].map(mapping)
df.head()

,id,first_name,last_name,email,gender,age,money,lon,lat,phone,registration,inactive,last_seen,address,preference,full_name,city,country
0,27625,Leandra,Pabelik,lpabelik5f@yale.edu,F,63,$55.18,18.284100,49.632552,0136319724,2019/04/16,False,1.559566e+09,"Palkovice, Czech Republic",entrée/plat/dessert,Leandra Pabelik,Palkovice,Czech Republic
1,27570,Ruthi,Ross,rross3w@sohu.com,F,57,$20.37,19.230220,50.466575,NaN,2018/10/23,False,1.567165e+09,"Siewierz, Poland",entrée/plat/dessert,Ruthi Ross,Siewierz,Poland
2,27572,Silas,Stourton,silas.stourton3y@answers.com,M,22,"€32,99",118.831081,24.984813,NaN,2018/12/30,False,1.553692e+09,"Longbo, China",entrée/plat/dessert,Silas Stourton,Longbo,China
3,27435,Roxine,Pettecrew,rpettecrew5@gravatar.com,F,64,"€98,93",121.648987,-8.844744,NaN,2019/03/12,False,1.552349e+09,"Potulando, Indonesia",entrée/plat,Roxine Pettecrew,Potulando,Indonesia
4,27558,Margaux,Gowanson,nope@thankyou.,F,54,$13.30,14.772557,45.160472,NaN,2018/08/13,False,1.543383e+09,"Bribir, Croatia",entrée/plat/dessert,Margaux Gowanson,Bribir,Croatia


In [54]:
# comportement de map()
df['gender'] = df['gender'].apply(lambda x: mapping.get(x, np.nan))

In [54]:
# dictionnaire incomplet
df['gender'] = df['gender'].apply(lambda x: mapping.get(x, x))

In [79]:
# traitement du gender, map() avec une Series
mapping = pd.Series({'Female': 'F', 'Male': 'M', 'F': 'F', 'M': 'M'})
mapping

Female    F
Male      M
F         F
M         M
dtype: object

In [57]:
pd.Series(['a', 'b', 'c'])

0    a
1    b
2    c
dtype: object

In [59]:
pd.DataFrame([['a', 'A'], ['b', 'B']])

,0,1
0,a,A
1,b,B


In [61]:
pd.DataFrame([{'a': 'A', 'b': 'C'}, {'b': 'B', 'a': 'D'}])

,a,b
0,A,C
1,D,B


In [80]:
# traitement du gender avec une Series
df['gender'] = df['gender'].map(mapping)

In [81]:
df['gender'].value_counts()

M    115
F     92
Name: gender, dtype: int64

#### Analyse du genre

In [12]:
# analyse gender NaN
len(df.loc[df['gender'].isnull()])

2

In [13]:
# analyse prénom avec gender NaN
df.loc[df['gender'].isnull(), 'first_name'].nunique()

0

In [14]:
# analyse gender
df.loc[df['gender'].isnull(), 'first_name'].unique()

array([nan], dtype=object)

Compléter le genre :

1. Autocomplétion avec le fichier people.csv (mais très peu de cas)
2. Gender API : https://gender-api.com/fr (simple mais API payante si gros volumes + de 500/mois)
3. US SSA baby names : https://www.ssa.gov/oact/babynames/limits.html ("gratuit", stats à produire, éventuellement affiner par année de naissance)

In [15]:
# 1) Fichier people.csv
for first_name in df.loc[df['gender'].isnull(), 'first_name'].unique():
    print(first_name, df.loc[df['first_name']==first_name, 'gender'].unique())

nan []


#### Analyse de l'âge

pandas.Series.astype() : types

pandas.to_numeric() : data avec gestion des erreurs

In [23]:
# analyse de l'âge
df.loc[df['age'].str.contains('[^0-9]'), 'age'].value_counts()

no       4
nope     2
sorry    2
Name: age, dtype: int64

In [25]:
# traitement de l'âge
df['age'] = pd.to_numeric(df['age'], errors='coerce')
df

,id,first_name,last_name,email,gender,age,money,lon,lat,phone,registration,inactive,last_seen,address,preference,full_name,city,country
0,27625,Leandra,Pabelik,lpabelik5f@yale.edu,Female,63.0,$55.18,18.284100,49.632552,0136319724,2019/04/16,False,1.559566e+09,"Palkovice, Czech Republic",entrée/plat/dessert,Leandra Pabelik,Palkovice,Czech Republic
1,27570,Ruthi,Ross,rross3w@sohu.com,Female,57.0,$20.37,19.230220,50.466575,NaN,2018/10/23,False,1.567165e+09,"Siewierz, Poland",entrée/plat/dessert,Ruthi Ross,Siewierz,Poland
2,27572,Silas,Stourton,silas.stourton3y@answers.com,Male,22.0,"€32,99",118.831081,24.984813,NaN,2018/12/30,False,1.553692e+09,"Longbo, China",entrée/plat/dessert,Silas Stourton,Longbo,China
3,27435,Roxine,Pettecrew,rpettecrew5@gravatar.com,F,64.0,"€98,93",121.648987,-8.844744,NaN,2019/03/12,False,1.552349e+09,"Potulando, Indonesia",entrée/plat,Roxine Pettecrew,Potulando,Indonesia
4,27558,Margaux,Gowanson,nope@thankyou.,Female,54.0,$13.30,14.772557,45.160472,NaN,2018/08/13,False,1.543383e+09,"Bribir, Croatia",entrée/plat/dessert,Margaux Gowanson,Bribir,Croatia
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
204,27465,Adelind,Christin,achristinz@blogs.com,Female,30.0,"€81,84",41.427853,52.415968,NaN,2019/06/04,False,1.559606e+09,"Znamenka, Russia",entrée/plat/dessert,Adelind Christin,Znamenka,Russia
205,27476,Petronella,Pickance,ppickance1a@uiuc.edu,Female,28.0,€37.15,132.702111,33.762296,NaN,2017/11/22,False,1.532870e+09,"Iyo, Japan",entrée/plat/dessert,Petronella Pickance,Iyo,Japan
206,27521,Moritz,Issacof,missacof2j@wired.com,Male,21.0,$91.97,72.311099,31.263396,0298949280,2018/08/20,False,1.549817e+09,"Jhang Sadr, Pakistan",entrée/plat/dessert,Moritz Issacof,Jhang Sadr,Pakistan
207,27441,Quintana,Foulstone,qfoulstoneb@newsvine.com,F,32.0,"€55,64",111.138442,-6.783653,0734595126,2018/10/25,False,1.540426e+09,"Mantingantengah, Indonesia",entrée/plat/dessert,Quintana Foulstone,Mantingantengah,Indonesia


#### Traitement des dates


pandas.to_datetime() : data, gestion des formats et des erreurs

pandas.Series.combine_first() : équivalent à fillna()

In [27]:
df['last_seen']

0      1.559566e+09
1      1.567165e+09
2      1.553692e+09
3      1.552349e+09
4      1.543383e+09
           ...     
204    1.559606e+09
205    1.532870e+09
206    1.549817e+09
207    1.540426e+09
208    1.540441e+09
Name: last_seen, Length: 203, dtype: float64

In [28]:
# conversion des dates
df['registration'] = pd.to_datetime(df.registration)
df['last_seen'] = pd.to_datetime(df.last_seen, unit='s')
# si last_seen est NaN, prendre registration
df['last_seen'] = df['last_seen'].fillna(df['registration'])
# idem
df['last_seen'] = df['last_seen'].combine_first(df['registration'])

In [29]:
df.head()

,id,first_name,last_name,email,gender,age,money,lon,lat,phone,registration,inactive,last_seen,address,preference,full_name,city,country
0,27625,Leandra,Pabelik,lpabelik5f@yale.edu,Female,63.0,$55.18,18.284100,49.632552,0136319724,2019-04-16,False,2019-06-03 12:39:57,"Palkovice, Czech Republic",entrée/plat/dessert,Leandra Pabelik,Palkovice,Czech Republic
1,27570,Ruthi,Ross,rross3w@sohu.com,Female,57.0,$20.37,19.230220,50.466575,NaN,2018-10-23,False,2019-08-30 11:41:04,"Siewierz, Poland",entrée/plat/dessert,Ruthi Ross,Siewierz,Poland
2,27572,Silas,Stourton,silas.stourton3y@answers.com,Male,22.0,"€32,99",118.831081,24.984813,NaN,2018-12-30,False,2019-03-27 13:10:32,"Longbo, China",entrée/plat/dessert,Silas Stourton,Longbo,China
3,27435,Roxine,Pettecrew,rpettecrew5@gravatar.com,F,64.0,"€98,93",121.648987,-8.844744,NaN,2019-03-12,False,2019-03-12 00:00:00,"Potulando, Indonesia",entrée/plat,Roxine Pettecrew,Potulando,Indonesia
4,27558,Margaux,Gowanson,nope@thankyou.,Female,54.0,$13.30,14.772557,45.160472,NaN,2018-08-13,False,2018-11-28 05:24:30,"Bribir, Croatia",entrée/plat/dessert,Margaux Gowanson,Bribir,Croatia


In [30]:
pd.Series.fillna?

#### Analyse de currency

In [31]:
# échantillon
np.random.seed(0)
df.sample(10)

,id,first_name,last_name,email,gender,age,money,lon,lat,phone,registration,inactive,last_seen,address,preference,full_name,city,country
18,27513,Dominick,Hryncewicz,dhryncewicz2b@amazonaws.com,Male,56.0,$80.71,68.214686,38.530112,NaN,2017-08-04,False,2019-07-26 02:33:01,"Tursunzoda, Tajikistan",entrée/plat/dessert,Dominick Hryncewicz,Tursunzoda,Tajikistan
45,27432,Gardener,Kempson,gkempson2@furl.net,M,33.0,$41.30,25.911060,49.103782,0774378059,2017-09-08,False,2019-03-18 18:58:59,"Kopychyntsi, Ukraine",entrée/plat/dessert,Gardener Kempson,Kopychyntsi,Ukraine
33,27463,Caesar,Ferrarotti,cferrarottix@qq.com,M,59.0,€60.84,112.867154,-6.921845,0346857462,2018-12-10,False,2019-10-02 02:43:57,"Trogan Barat, Indonesia",entrée/plat/dessert,Caesar Ferrarotti,Trogan Barat,Indonesia
37,27617,Smitty,Richter,srichter57@pinterest.com,Male,54.0,$19.07,37.149279,56.747788,0789,2018-10-20,False,2019-04-14 11:57:19,"Zarya, Russia",entrée/plat/dessert,Smitty Richter,Zarya,Russia
112,27544,Cristiano,Midghall,cmidghall36@npr.org,Male,47.0,€43.94,-75.649902,4.827161,0574769298,2018-01-25,False,2019-06-15 14:02:49,"Dos Quebradas, Colombia",entrée/plat/dessert,Cristiano Midghall,Dos Quebradas,Colombia
91,27618,Murielle,Beston,mbeston58@youku.com,Female,38.0,"€99,47",173.138978,1.386973,NaN,2017-02-19,False,2018-09-10 16:16:38,"Bonriki Village, Kiribati",entrée/plat/dessert,Murielle Beston,Bonriki Village,Kiribati
5,27497,Gordie,Bodicum,gbodicum1v@apache.org,Male,60.0,$47.26,13.978681,49.786243,0146943857,2017-04-29,False,2018-12-26 07:40:36,"Jince, Czech Republic",entrée/plat/dessert,Gordie Bodicum,Jince,Czech Republic
127,27431,Sinclair,Witherow,switherow1@joomla.org,M,41.0,€55.95,22.004971,56.674534,0298040087,2018-12-26,False,2018-12-26 00:00:00,"Skrunda, Latvia",entrée/plat/dessert,Sinclair Witherow,Skrunda,Latvia
12,27548,Stanley,McCullen,smccullen3a@timesonline.co.uk,Male,60.0,NaN,119.141473,28.074649,0875503094,2017-06-05,False,2017-06-05 00:00:00,"Longquan, China",entrée/plat/dessert,Stanley McCullen,Longquan,China
157,27526,Sarajane,Fidler,sfidler2o@sitemeter.com,Female,70.0,$90.09,95.308054,32.596752,0511530696,2017-04-16,False,2018-08-09 11:29:14,"Jieduo, China",entrée/plat/dessert,Sarajane Fidler,Jieduo,China


#### Traitement de 'currency'

Produire une nouvelle colonne numérique 'money_eur'.

Pour la conversion USD/EUR, on utilise l'API https://api.exchangeratesapi.io/latest

In [32]:
# API
import json

import requests

response = requests.get('https://api.exchangeratesapi.io/latest')
rates = json.loads(response.content)
rates

{'rates': {'CAD': 1.5497,
  'HKD': 9.0993,
  'ISK': 163.2,
  'PHP': 57.148,
  'DKK': 7.442,
  'HUF': 364.36,
  'CZK': 27.279,
  'AUD': 1.6555,
  'RON': 4.8763,
  'SEK': 10.3593,
  'IDR': 17317.86,
  'INR': 86.0715,
  'BRL': 6.6054,
  'RUB': 91.5767,
  'HRK': 7.582,
  'JPY': 123.59,
  'THB': 36.608,
  'CHF': 1.0721,
  'SGD': 1.5942,
  'PLN': 4.5558,
  'BGN': 1.9558,
  'TRY': 9.33,
  'CNY': 7.8639,
  'NOK': 10.957,
  'NZD': 1.7746,
  'ZAR': 19.3867,
  'USD': 1.1741,
  'MXN': 24.872,
  'ILS': 3.9673,
  'GBP': 0.90915,
  'KRW': 1341.16,
  'MYR': 4.8708},
 'base': 'EUR',
 'date': '2020-10-16'}

In [33]:
df['money'].value_counts()

$42.44    3
$94.45    1
$40.93    1
$77.78    1
€18,67    1
         ..
€90.13    1
$95.51    1
$79.67    1
€77.18    1
€70.24    1
Name: money, Length: 184, dtype: int64

In [40]:
df['currency'] = df['money'].str[0].map({'€': 'EUR', '$': 'USD'})
df.head()

,id,first_name,last_name,email address,gender,age,money,lon,lat,phone,registration,inactive,last_seen,address,preference,currency
0,27625,Leandra,Pabelik,lpabelik5f@yale.edu,Female,63,$55.18,18.284100,49.632552,0136319724,2019/04/16,False,1.559566e+09,"Palkovice, Czech Republic",entrée/plat/dessert,USD
1,27570,Ruthi,Ross,rross3w@sohu.com,Female,57,$20.37,19.230220,50.466575,NaN,2018/10/23,False,1.567165e+09,"Siewierz, Poland",entrée/plat/dessert,USD
2,27572,Silas,Stourton,silas.stourton3y@answers.com,Male,22,"€32,99",118.831081,24.984813,NaN,2018/12/30,False,1.553692e+09,"Longbo, China",entrée/plat/dessert,EUR
3,27435,Roxine,Pettecrew,rpettecrew5@gravatar.com,F,64,"£98,93",121.648987,-8.844744,NaN,2019/03/12,False,1.552349e+09,"Potulando, Indonesia",entrée/plat,NaN
4,27558,Margaux,Gowanson,nope@thankyou.,Female,54,$13.30,14.772557,45.160472,NaN,2018/08/13,False,1.543383e+09,"Bribir, Croatia",entrée/plat/dessert,USD


In [36]:
rates['rates']

{'CAD': 1.5497,
 'HKD': 9.0993,
 'ISK': 163.2,
 'PHP': 57.148,
 'DKK': 7.442,
 'HUF': 364.36,
 'CZK': 27.279,
 'AUD': 1.6555,
 'RON': 4.8763,
 'SEK': 10.3593,
 'IDR': 17317.86,
 'INR': 86.0715,
 'BRL': 6.6054,
 'RUB': 91.5767,
 'HRK': 7.582,
 'JPY': 123.59,
 'THB': 36.608,
 'CHF': 1.0721,
 'SGD': 1.5942,
 'PLN': 4.5558,
 'BGN': 1.9558,
 'TRY': 9.33,
 'CNY': 7.8639,
 'NOK': 10.957,
 'NZD': 1.7746,
 'ZAR': 19.3867,
 'USD': 1.1741,
 'MXN': 24.872,
 'ILS': 3.9673,
 'GBP': 0.90915,
 'KRW': 1341.16,
 'MYR': 4.8708}

In [42]:
# extraction de la currency
df['currency'] = df['money'].str[0].map({'€': 'EUR', '$': 'USD', '£': 'GBP'})
df['money_eur'] = df['money'].str[1:].str.replace(',', '.')  # extraction des derniers chars + , => .
df['money_eur'] = pd.to_numeric(df['money_eur'])  # conversion en nombre

# conversion des monnaies en euros
rates['rates']['EUR'] = 1.0
df['money_eur'] = df['money_eur'] / df['currency'].map(rates['rates'])
#np.random.seed(0)
#df.sample(10)
df.head()

,id,first_name,last_name,email address,gender,age,money,lon,lat,phone,registration,inactive,last_seen,address,preference,currency,money_eur
0,27625,Leandra,Pabelik,lpabelik5f@yale.edu,Female,63,$55.18,18.284100,49.632552,0136319724,2019/04/16,False,1.559566e+09,"Palkovice, Czech Republic",entrée/plat/dessert,USD,46.997700
1,27570,Ruthi,Ross,rross3w@sohu.com,Female,57,$20.37,19.230220,50.466575,NaN,2018/10/23,False,1.567165e+09,"Siewierz, Poland",entrée/plat/dessert,USD,17.349459
2,27572,Silas,Stourton,silas.stourton3y@answers.com,Male,22,"€32,99",118.831081,24.984813,NaN,2018/12/30,False,1.553692e+09,"Longbo, China",entrée/plat/dessert,EUR,32.990000
3,27435,Roxine,Pettecrew,rpettecrew5@gravatar.com,F,64,"£98,93",121.648987,-8.844744,NaN,2019/03/12,False,1.552349e+09,"Potulando, Indonesia",entrée/plat,GBP,108.815927
4,27558,Margaux,Gowanson,nope@thankyou.,Female,54,$13.30,14.772557,45.160472,NaN,2018/08/13,False,1.543383e+09,"Bribir, Croatia",entrée/plat/dessert,USD,11.327826


#### Analyse des emails

On va utiliser des regex pour nettoyer les emails mais mieux vaut utiliser une librairie spécialisée. Par exemple, https://github.com/syrusakbary/validate_email

In [45]:
# email NaN
df['email'].isnull().sum()

4

In [46]:
# suppression des emails absents
df = df.dropna(subset=['email']).copy()

In [47]:
len(df)

203

In [48]:
# emails avec chars non admis
df.loc[df['email'].str.contains('[^A-Za-z0-9_\-%+.@]'), 'email'].values

array(['ycurston4a@nifty.com  ', '  agilmartin4b@un.org ',
       '  swinchcum4o@apple.com', '  erowcliffe43@chronoengine.com',
       'rbellenie54@topsy.com ', 'dbrindle1p@trellian.com ',
       ' efeatonby4i@hc360.com', ' mlowcock4u@nba.com'], dtype=object)

In [49]:
# suppression des blancs
df['email'] = df['email'].str.strip()
df.loc[df['email'].str.contains('[^A-Za-z0-9_\-%+.@]'), 'email']

Series([], Name: email, dtype: object)

In [61]:
with open('people.csv', encoding='latin-1') as f:
    for line in f:
        print(line)
        break

id,first_name,last_name,email address,gender,age,money,lon,lat,phone,registration,inactive,last_seen,address,preference



In [62]:
line

'id,first_name,last_name,email address,gender,age,money,lon,lat,phone,registration,inactive,last_seen,address,preference\n'

In [58]:
line.rstrip()

'id,first_name,last_name,email address,gender,age,money,lon,lat,phone,registration,inactive,last_seen,address,preference'

In [67]:
# regex pour vérifier les domaines
df.loc[~df['email'].str.contains('.+@[A-Za-z0-9_\-.]+\.[A-Za-z]{2,}')]

,id,first_name,last_name,email,gender,age,money,lon,lat,phone,registration,inactive,last_seen,address,preference,currency,money_eur


In [64]:
# emails avec noms de domaine invalides
df = df.loc[df['email'].str.contains('.+@[A-Za-z0-9_\-.]+\.[A-Za-z]{2,}')]

In [68]:
# emails avec aliases (char +)
df.loc[df['email'].str.contains('\+'), 'email'].values

array(['a.gorz+alias@gmail.com'], dtype=object)

In [69]:
# extraction des aliases (char +)
tab_email = df['email'].str.extract('([^+]+)(\+.*)?(@.+)').sort_values(0)
tab_email

,0,1,2
89,Izzy.Clemence45,NaN,@hhs.gov
186,a.gorz,+alias,@gmail.com
185,a.gorz,NaN,@gmail.com
40,aMatthisson3s,NaN,@ted.com
39,abonar23,NaN,@wufoo.com
...,...,...,...
159,wconcannon2l,NaN,@merriam-webster.com
53,wshewery11,NaN,@smugmug.com
74,ycurston4a,NaN,@nifty.com
80,zreinhard2f,NaN,@github.com


In [ ]:
# '^...' regex au début

In [ ]:
# [abc]
# [a-zA-Z]
# [^abc] : pas a, b ou c
# [^0-9] : pas digit
# '<[^>]+>' : tag HTML
# '<.+>'

In [ ]:
# caractères parenthèses
# \([0-9]+\)

In [ ]:
# parenthèses non capturante
# (?:regex)

In [70]:
# suprresion des aliases (char +)
df['email'] = tab_email[0] + tab_email[2]
df.sort_values('email')

,id,first_name,last_name,email,gender,age,money,lon,lat,phone,registration,inactive,last_seen,address,preference,currency,money_eur
89,27579,Izzy,Clemence,Izzy.Clemence45@hhs.gov,Male,48,€46.75,97.521763,40.324768,NaN,2019/08/10,False,1.565395e+09,"Xigebi, China",entrée/plat/dessert,EUR,46.750000
186,27902,Goran,Drez,a.gorz@gmail.com,Male,55,NaN,88.166695,39.024258,NaN,2019/07/23,False,NaN,"Paris, France",entrée/plat/dessert,NaN,NaN
185,27596,André,Gorz,a.gorz@gmail.com,Male,43,€17.20,88.166695,39.024258,NaN,2019/05/23,False,1.570824e+09,"Vosnon, France",entrée/plat/dessert,EUR,17.200000
40,27566,Amalie,Matthisson,aMatthisson3s@ted.com,Female,63,"€13,89",16.304150,57.993607,0409272683,2019/01/20,False,1.547942e+09,"Linköping, Sweden",entrée/plat/dessert,EUR,13.890000
39,27505,Alvis,Bonar,abonar23@wufoo.com,Male,27,$90.88,14.426644,49.660577,NaN,2017/03/09,False,1.552137e+09,"Sedlčany, Czech Republic",entrée/plat/dessert,USD,77.403969
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
159,27523,Wolfgang,Concannon,wconcannon2l@merriam-webster.com,Male,59,€51.68,120.174895,30.387552,NaN,2017/07/10,False,1.570378e+09,"Chongxian, China",entrée/plat/dessert,EUR,51.680000
53,27467,Worden,Shewery,wshewery11@smugmug.com,Male,26,$15.67,19.883333,40.916667,0517301167,2018/08/22,False,1.542873e+09,"Kajan, Albania",entrée/plat/dessert,USD,13.346393
74,27584,Yehudit,Curston,ycurston4a@nifty.com,Male,63,$86.99,-106.417031,31.975476,0157438239,2019/01/09,False,1.546992e+09,"El Paso, United States",entrée/plat/dessert,USD,74.090793
80,27517,Zack,Reinhard,zreinhard2f@github.com,Male,35,$91.20,-68.103824,-38.961513,0056565079,2017/10/20,False,1.555830e+09,"Las Lajas, Argentina",entrée/plat/dessert,USD,77.676518


In [71]:
# suppression des emails en double, on conserve la première ligne
df = df.drop_duplicates(subset=['email'])
df.sort_values('email')

,id,first_name,last_name,email,gender,age,money,lon,lat,phone,registration,inactive,last_seen,address,preference,currency,money_eur
89,27579,Izzy,Clemence,Izzy.Clemence45@hhs.gov,Male,48,€46.75,97.521763,40.324768,NaN,2019/08/10,False,1.565395e+09,"Xigebi, China",entrée/plat/dessert,EUR,46.750000
185,27596,André,Gorz,a.gorz@gmail.com,Male,43,€17.20,88.166695,39.024258,NaN,2019/05/23,False,1.570824e+09,"Vosnon, France",entrée/plat/dessert,EUR,17.200000
40,27566,Amalie,Matthisson,aMatthisson3s@ted.com,Female,63,"€13,89",16.304150,57.993607,0409272683,2019/01/20,False,1.547942e+09,"Linköping, Sweden",entrée/plat/dessert,EUR,13.890000
39,27505,Alvis,Bonar,abonar23@wufoo.com,Male,27,$90.88,14.426644,49.660577,NaN,2017/03/09,False,1.552137e+09,"Sedlčany, Czech Republic",entrée/plat/dessert,USD,77.403969
204,27465,Adelind,Christin,achristinz@blogs.com,Female,30,"€81,84",41.427853,52.415968,NaN,2019/06/04,False,1.559606e+09,"Znamenka, Russia",entrée/plat/dessert,EUR,81.840000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
159,27523,Wolfgang,Concannon,wconcannon2l@merriam-webster.com,Male,59,€51.68,120.174895,30.387552,NaN,2017/07/10,False,1.570378e+09,"Chongxian, China",entrée/plat/dessert,EUR,51.680000
53,27467,Worden,Shewery,wshewery11@smugmug.com,Male,26,$15.67,19.883333,40.916667,0517301167,2018/08/22,False,1.542873e+09,"Kajan, Albania",entrée/plat/dessert,USD,13.346393
74,27584,Yehudit,Curston,ycurston4a@nifty.com,Male,63,$86.99,-106.417031,31.975476,0157438239,2019/01/09,False,1.546992e+09,"El Paso, United States",entrée/plat/dessert,USD,74.090793
80,27517,Zack,Reinhard,zreinhard2f@github.com,Male,35,$91.20,-68.103824,-38.961513,0056565079,2017/10/20,False,1.555830e+09,"Las Lajas, Argentina",entrée/plat/dessert,USD,77.676518


#### Analyse de la colonne 'preference'

In [72]:
# analyse de preference
df['preference'].nunique()

5

In [73]:
# analyse de preference
df['preference'].value_counts()

entrée/plat/dessert    180
plat/dessert             4
plat                     3
entrée/plat              3
boisson                  1
Name: preference, dtype: int64

In [74]:
# modalités de preference
s = set()
df['preference'].apply(lambda x: s.update(x.split('/')))
s

{'boisson', 'dessert', 'entrée', 'plat'}

In [77]:
# modalités de preference
l = list()
df['preference'].apply(lambda x: l.extend(x.split('/')))
l

['entrée',
 'plat',
 'dessert',
 'entrée',
 'plat',
 'dessert',
 'entrée',
 'plat',
 'dessert',
 'entrée',
 'plat',
 'entrée',
 'plat',
 'dessert',
 'entrée',
 'plat',
 'dessert',
 'entrée',
 'plat',
 'dessert',
 'entrée',
 'plat',
 'dessert',
 'entrée',
 'plat',
 'dessert',
 'entrée',
 'plat',
 'dessert',
 'entrée',
 'plat',
 'dessert',
 'entrée',
 'plat',
 'dessert',
 'entrée',
 'plat',
 'dessert',
 'entrée',
 'plat',
 'dessert',
 'entrée',
 'plat',
 'entrée',
 'plat',
 'dessert',
 'entrée',
 'plat',
 'dessert',
 'entrée',
 'plat',
 'dessert',
 'entrée',
 'plat',
 'dessert',
 'entrée',
 'plat',
 'dessert',
 'entrée',
 'plat',
 'dessert',
 'entrée',
 'plat',
 'dessert',
 'entrée',
 'plat',
 'dessert',
 'entrée',
 'plat',
 'dessert',
 'entrée',
 'plat',
 'dessert',
 'entrée',
 'plat',
 'dessert',
 'entrée',
 'plat',
 'dessert',
 'entrée',
 'plat',
 'entrée',
 'plat',
 'dessert',
 'entrée',
 'plat',
 'dessert',
 'entrée',
 'plat',
 'dessert',
 'entrée',
 'plat',
 'dessert',
 'entrée',
 

In [75]:
# ajour d'un booléen par preference
for x in sorted(s):
    df[x] = df['preference'].str.contains(x)
    
df

,id,first_name,last_name,email,gender,age,money,lon,lat,phone,...,inactive,last_seen,address,preference,currency,money_eur,boisson,dessert,entrée,plat
0,27625,Leandra,Pabelik,lpabelik5f@yale.edu,Female,63,$55.18,18.284100,49.632552,0136319724,...,False,1.559566e+09,"Palkovice, Czech Republic",entrée/plat/dessert,USD,46.997700,False,True,True,True
1,27570,Ruthi,Ross,rross3w@sohu.com,Female,57,$20.37,19.230220,50.466575,NaN,...,False,1.567165e+09,"Siewierz, Poland",entrée/plat/dessert,USD,17.349459,False,True,True,True
2,27572,Silas,Stourton,silas.stourton3y@answers.com,Male,22,"€32,99",118.831081,24.984813,NaN,...,False,1.553692e+09,"Longbo, China",entrée/plat/dessert,EUR,32.990000,False,True,True,True
3,27435,Roxine,Pettecrew,rpettecrew5@gravatar.com,F,64,"£98,93",121.648987,-8.844744,NaN,...,False,1.552349e+09,"Potulando, Indonesia",entrée/plat,GBP,108.815927,False,False,True,True
5,27497,Gordie,Bodicum,gbodicum1v@apache.org,Male,60,$47.26,13.978681,49.786243,0146943857,...,False,1.545810e+09,"Jince, Czech Republic",entrée/plat/dessert,USD,40.252108,False,True,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
204,27465,Adelind,Christin,achristinz@blogs.com,Female,30,"€81,84",41.427853,52.415968,NaN,...,False,1.559606e+09,"Znamenka, Russia",entrée/plat/dessert,EUR,81.840000,False,True,True,True
205,27476,Petronella,Pickance,ppickance1a@uiuc.edu,Female,28,€37.15,132.702111,33.762296,NaN,...,False,1.532870e+09,"Iyo, Japan",entrée/plat/dessert,EUR,37.150000,False,True,True,True
206,27521,Moritz,Issacof,missacof2j@wired.com,Male,21,$91.97,72.311099,31.263396,0298949280,...,False,1.549817e+09,"Jhang Sadr, Pakistan",entrée/plat/dessert,USD,78.332340,False,True,True,True
207,27441,Quintana,Foulstone,qfoulstoneb@newsvine.com,F,32,"€55,64",111.138442,-6.783653,0734595126,...,False,1.540426e+09,"Mantingantengah, Indonesia",entrée/plat/dessert,EUR,55.640000,False,True,True,True


In [78]:
# autre façon avec get_dummies
df['preference'].str.get_dummies(sep='/')

,boisson,dessert,entrée,plat
0,0,1,1,1
1,0,1,1,1
2,0,1,1,1
3,0,0,1,1
5,0,1,1,1
...,...,...,...,...
204,0,1,1,1
205,0,1,1,1
206,0,1,1,1
207,0,1,1,1


In [79]:
# assignation des préférences
tab_preference = df['preference'].str.get_dummies(sep='/')
df[tab_preference.columns] = tab_preference.astype(bool)
df

,id,first_name,last_name,email,gender,age,money,lon,lat,phone,...,inactive,last_seen,address,preference,currency,money_eur,boisson,dessert,entrée,plat
0,27625,Leandra,Pabelik,lpabelik5f@yale.edu,Female,63,$55.18,18.284100,49.632552,0136319724,...,False,1.559566e+09,"Palkovice, Czech Republic",entrée/plat/dessert,USD,46.997700,False,True,True,True
1,27570,Ruthi,Ross,rross3w@sohu.com,Female,57,$20.37,19.230220,50.466575,NaN,...,False,1.567165e+09,"Siewierz, Poland",entrée/plat/dessert,USD,17.349459,False,True,True,True
2,27572,Silas,Stourton,silas.stourton3y@answers.com,Male,22,"€32,99",118.831081,24.984813,NaN,...,False,1.553692e+09,"Longbo, China",entrée/plat/dessert,EUR,32.990000,False,True,True,True
3,27435,Roxine,Pettecrew,rpettecrew5@gravatar.com,F,64,"£98,93",121.648987,-8.844744,NaN,...,False,1.552349e+09,"Potulando, Indonesia",entrée/plat,GBP,108.815927,False,False,True,True
5,27497,Gordie,Bodicum,gbodicum1v@apache.org,Male,60,$47.26,13.978681,49.786243,0146943857,...,False,1.545810e+09,"Jince, Czech Republic",entrée/plat/dessert,USD,40.252108,False,True,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
204,27465,Adelind,Christin,achristinz@blogs.com,Female,30,"€81,84",41.427853,52.415968,NaN,...,False,1.559606e+09,"Znamenka, Russia",entrée/plat/dessert,EUR,81.840000,False,True,True,True
205,27476,Petronella,Pickance,ppickance1a@uiuc.edu,Female,28,€37.15,132.702111,33.762296,NaN,...,False,1.532870e+09,"Iyo, Japan",entrée/plat/dessert,EUR,37.150000,False,True,True,True
206,27521,Moritz,Issacof,missacof2j@wired.com,Male,21,$91.97,72.311099,31.263396,0298949280,...,False,1.549817e+09,"Jhang Sadr, Pakistan",entrée/plat/dessert,USD,78.332340,False,True,True,True
207,27441,Quintana,Foulstone,qfoulstoneb@newsvine.com,F,32,"€55,64",111.138442,-6.783653,0734595126,...,False,1.540426e+09,"Mantingantengah, Indonesia",entrée/plat/dessert,EUR,55.640000,False,True,True,True


#### clean

In [81]:
import json

import requests
import pandas as pd

def clean_people(df):
    
    # suppression des lignes dupliquées
    df = df.drop_duplicates()
    
    # renommer les colonnes
    df = df.rename(columns={'email address': 'email'})
    
    # supprimer uniquement les lignes dont le first_name vaut NaN 
    df = df.dropna(subset=['first_name'])
    
    # ajout d'une colonne 'full_name'
    df['full_name'] = df['first_name'] + ' ' + df['last_name']

    # calcul de city et country à partir de address
    df[['city', 'country']] = df['address'].str.extract('(.*), (.*)')

    # traitement du gender
    mapping = {'Female': 'F', 'Male': 'M'}
    df['gender'] = df['gender'].map(mapping)
 
    # traitement de l'âge
    df['age'] = pd.to_numeric(df['age'], errors='coerce')

    # conversion des dates
    df['registration'] = pd.to_datetime(df.registration)
    df['last_seen'] = pd.to_datetime(df.last_seen, unit='s')
    # si last_seen est NaN, prendre registration
    df['last_seen'] = df['last_seen'].fillna(df['registration'])

    # récupération des taux de change
    response = requests.get('https://api.exchangeratesapi.io/latest')
    rates = json.loads(response.content)

    # extraction de la currency
    df['currency'] = df['money'].str[0].map({'€': 'EUR', '$': 'USD'})
    df['money_eur'] = df['money'].str[1:].str.replace(',', '.')  # extraction des derniers chars + , => .
    df['money_eur'] = pd.to_numeric(df['money_eur'])  # conversion en nombre

    # conversion des monnaies en euros
    rates['rates']['EUR'] = 1.0  # ajour de EUR pour pouvoir utiliser map()
    df['money_eur'] = df['money_eur'] * df['currency'].map(rates['rates'])

    # suppression des emails absents
    df = df.dropna(subset=['email'])

    # suppression des blancs
    df['email'] = df['email'].str.strip()

    # emails avec noms de domaine valides
    df = df.loc[df['email'].str.contains('.+@[A-Za-z0-9_\-.]+\.[A-Za-z]{2,}')]
    
    # extraction des aliases (char +)
    tab_email = df['email'].str.extract('([^+]+)(\+.*)?(@.+)').sort_values(0)
    # suppression des aliases (char +)
    df['email'] = tab_email[0] + tab_email[2]

    # suppression des emails en double, on conserve la première ligne
    df = df.drop_duplicates(subset=['email'])

    # assignation des préférences
    tab_preference = df['preference'].str.get_dummies(sep='/')
    df[tab_preference.columns] = tab_preference.astype(bool)

    return df

In [82]:
# run
import pandas as pd

df = pd.read_csv('people.csv')
print(df.shape)

df = clean_people(df)
print(df.shape)

(209, 15)
(191, 24)


**Exercice**

1. Téléchargez le fichier Excel "FranceTHD_Open_Data_Observatoire_2017_T2.xlsx" sur le niveau des débits sur les réseaux d'accès à Internet : ADSL, câble, Fibre FttH (T2 2015 - T2 2017) de la page : https://www.data.gouv.fr/fr/datasets/niveau-des-debits-sur-les-reseaux-dacces-a-internet-adsl-cable-fibre-ftth-t2-2015-t2-2017/

2. Chargez avec pd.read_excel() dans un DataFrame le dernier onglet "2017 T2" en mesurant le temps avec %%time en première instruction de cellule.

3. Modifiez le nom des 4 premières colonnes en : 'code INSEE', 'commune', 'département', 'nb locaux' par exemple.

4. Sauvegardez le DataFrame avec pd.to_pickle().

5. Rechargez le DataFrame à partir du fichier pickle en mesurant le temps avec %%time en première instruction de cellule et comparez.

6. Effectuez une opération de sélection sur les communes : par exemple, les communes qui commencent par "SAINT".

7. Diagnostiquez le message d'erreur.

8. Corrigez le DataFrame.

In [84]:
%%time
df = pd.read_excel('FranceTHD_Open_Data_Observatoire_2017_T2.xlsx',
               sheet_name=-1,
               header=1)
df

Wall time: 1min 41s


,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,éligibles,3M et +,8M et +,30M et +,100M et +,éligibles.1,...,éligibles.2,3M et +.2,8M et +.2,30M et +.2,100M et +.2,éligibles.3,3M et +.3,8M et +.3,30M et +.3,100M et +.3
0,01001,L'Abergement-Clémenciat,01,361,1.000,0.456,0.052,0.000,0.000,1.000,...,0.0,0.0,0.0,0.0,0.0,0.000,0.000,0.000,0.000,0.000
1,01002,L'Abergement-de-Varey,01,168,0.288,0.076,0.024,0.024,0.024,0.276,...,0.0,0.0,0.0,0.0,0.0,0.024,0.024,0.024,0.024,0.024
2,01004,Ambérieu-en-Bugey,01,7501,0.998,0.974,0.898,0.430,0.198,0.998,...,0.0,0.0,0.0,0.0,0.0,0.198,0.198,0.198,0.198,0.198
3,01005,Ambérieux-en-Dombes,01,725,1.000,0.993,0.945,0.667,0.036,1.000,...,0.0,0.0,0.0,0.0,0.0,0.036,0.036,0.036,0.036,0.036
4,01006,Ambléon,01,73,1.000,1.000,1.000,1.000,1.000,1.000,...,0.0,0.0,0.0,0.0,0.0,1.000,1.000,1.000,1.000,1.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36641,97615,Pamandzi,976,2670,1.000,0.998,0.994,0.564,0.000,1.000,...,0.0,0.0,0.0,0.0,0.0,0.000,0.000,0.000,0.000,0.000
36642,97616,Sada,976,2690,0.991,0.965,0.839,0.153,0.000,0.991,...,0.0,0.0,0.0,0.0,0.0,0.000,0.000,0.000,0.000,0.000
36643,97617,Tsingoni,976,3099,0.987,0.708,0.706,0.475,0.000,0.987,...,0.0,0.0,0.0,0.0,0.0,0.000,0.000,0.000,0.000,0.000
36644,97701,Saint-Barthélemy,977,4693,1.000,0.936,0.762,0.146,0.000,1.000,...,0.0,0.0,0.0,0.0,0.0,0.000,0.000,0.000,0.000,0.000


In [86]:
df.columns = ['code INSEE', 'commune', 'département', 'nb locaux'] \
            + list(df.columns[4:])
df.head()

,code INSEE,commune,département,nb locaux,éligibles,3M et +,8M et +,30M et +,100M et +,éligibles.1,...,éligibles.2,3M et +.2,8M et +.2,30M et +.2,100M et +.2,éligibles.3,3M et +.3,8M et +.3,30M et +.3,100M et +.3
0,01001,L'Abergement-Clémenciat,01,361,1.000,0.456,0.052,0.000,0.000,1.000,...,0.0,0.0,0.0,0.0,0.0,0.000,0.000,0.000,0.000,0.000
1,01002,L'Abergement-de-Varey,01,168,0.288,0.076,0.024,0.024,0.024,0.276,...,0.0,0.0,0.0,0.0,0.0,0.024,0.024,0.024,0.024,0.024
2,01004,Ambérieu-en-Bugey,01,7501,0.998,0.974,0.898,0.430,0.198,0.998,...,0.0,0.0,0.0,0.0,0.0,0.198,0.198,0.198,0.198,0.198
3,01005,Ambérieux-en-Dombes,01,725,1.000,0.993,0.945,0.667,0.036,1.000,...,0.0,0.0,0.0,0.0,0.0,0.036,0.036,0.036,0.036,0.036
4,01006,Ambléon,01,73,1.000,1.000,1.000,1.000,1.000,1.000,...,0.0,0.0,0.0,0.0,0.0,1.000,1.000,1.000,1.000,1.000


In [87]:
df.to_pickle('thd.dat')

In [88]:
%%time
df = pd.read_pickle('thd.dat')
df.head()

Wall time: 35 ms


,code INSEE,commune,département,nb locaux,éligibles,3M et +,8M et +,30M et +,100M et +,éligibles.1,...,éligibles.2,3M et +.2,8M et +.2,30M et +.2,100M et +.2,éligibles.3,3M et +.3,8M et +.3,30M et +.3,100M et +.3
0,01001,L'Abergement-Clémenciat,01,361,1.000,0.456,0.052,0.000,0.000,1.000,...,0.0,0.0,0.0,0.0,0.0,0.000,0.000,0.000,0.000,0.000
1,01002,L'Abergement-de-Varey,01,168,0.288,0.076,0.024,0.024,0.024,0.276,...,0.0,0.0,0.0,0.0,0.0,0.024,0.024,0.024,0.024,0.024
2,01004,Ambérieu-en-Bugey,01,7501,0.998,0.974,0.898,0.430,0.198,0.998,...,0.0,0.0,0.0,0.0,0.0,0.198,0.198,0.198,0.198,0.198
3,01005,Ambérieux-en-Dombes,01,725,1.000,0.993,0.945,0.667,0.036,1.000,...,0.0,0.0,0.0,0.0,0.0,0.036,0.036,0.036,0.036,0.036
4,01006,Ambléon,01,73,1.000,1.000,1.000,1.000,1.000,1.000,...,0.0,0.0,0.0,0.0,0.0,1.000,1.000,1.000,1.000,1.000


In [90]:
df.loc[df['commune'].str.startswith('Saint')]

ValueError: cannot mask with array containing NA / NaN values

In [92]:
df.loc[df['commune'].isnull()].empty

True

In [93]:
df[df["commune"].str.startswith('Saint', na=False)]

,code INSEE,commune,département,nb locaux,éligibles,3M et +,8M et +,30M et +,100M et +,éligibles.1,...,éligibles.2,3M et +.2,8M et +.2,30M et +.2,100M et +.2,éligibles.3,3M et +.3,8M et +.3,30M et +.3,100M et +.3
298,01331,Saint-Alban,01,101,1.000,0.733,0.554,0.000,0.000,1.000,...,0.000,0.000,0.000,0.000,0.0,0.000,0.000,0.000,0.000,0.000
299,01332,Saint-André-de-Bâgé,01,289,1.000,1.000,0.882,0.071,0.021,1.000,...,0.000,0.000,0.000,0.000,0.0,0.021,0.021,0.021,0.021,0.021
300,01333,Saint-André-de-Corcy,01,1453,1.000,0.987,0.975,0.522,0.048,1.000,...,0.000,0.000,0.000,0.000,0.0,0.048,0.048,0.048,0.048,0.048
301,01334,Saint-André-d'Huiriat,01,256,1.000,0.942,0.881,0.819,0.819,1.000,...,0.000,0.000,0.000,0.000,0.0,0.819,0.819,0.819,0.819,0.819
302,01335,Saint-André-le-Bouchoux,01,143,0.986,0.007,0.007,0.007,0.007,0.986,...,0.000,0.000,0.000,0.000,0.0,0.007,0.007,0.007,0.007,0.007
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36619,97419,Sainte-Rose,974,2723,0.999,0.960,0.838,0.396,0.000,0.999,...,0.000,0.000,0.000,0.000,0.0,0.000,0.000,0.000,0.000,0.000
36620,97420,Sainte-Suzanne,974,8901,0.995,0.958,0.936,0.776,0.722,0.995,...,0.000,0.000,0.000,0.000,0.0,0.722,0.722,0.722,0.722,0.722
36626,97502,Saint-Pierre,975,2653,0.991,0.991,0.991,0.991,0.000,0.000,...,0.991,0.991,0.991,0.991,0.0,0.000,0.000,0.000,0.000,0.000
36644,97701,Saint-Barthélemy,977,4693,1.000,0.936,0.762,0.146,0.000,1.000,...,0.000,0.000,0.000,0.000,0.0,0.000,0.000,0.000,0.000,0.000


In [96]:
df['commune'].apply(type).value_counts()

<class 'str'>     36644
<class 'bool'>        2
Name: commune, dtype: int64

In [97]:
isinstance('toto', str)

True

In [98]:
df.loc[df['commune'].apply(lambda x: isinstance(x, bool))]

,code INSEE,commune,département,nb locaux,éligibles,3M et +,8M et +,30M et +,100M et +,éligibles.1,...,éligibles.2,3M et +.2,8M et +.2,30M et +.2,100M et +.2,éligibles.3,3M et +.3,8M et +.3,30M et +.3,100M et +.3
2579,08165,False,08,26,1.0,0.8,0.000,0.000,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8439,24177,False,24,384,1.0,1.0,0.991,0.617,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [99]:
df.loc[df['commune'].apply(lambda x: isinstance(x, bool)), 'commune'] = 'Faux'

In [100]:
df.loc[df['commune'].str.startswith('Saint')]

,code INSEE,commune,département,nb locaux,éligibles,3M et +,8M et +,30M et +,100M et +,éligibles.1,...,éligibles.2,3M et +.2,8M et +.2,30M et +.2,100M et +.2,éligibles.3,3M et +.3,8M et +.3,30M et +.3,100M et +.3
298,01331,Saint-Alban,01,101,1.000,0.733,0.554,0.000,0.000,1.000,...,0.000,0.000,0.000,0.000,0.0,0.000,0.000,0.000,0.000,0.000
299,01332,Saint-André-de-Bâgé,01,289,1.000,1.000,0.882,0.071,0.021,1.000,...,0.000,0.000,0.000,0.000,0.0,0.021,0.021,0.021,0.021,0.021
300,01333,Saint-André-de-Corcy,01,1453,1.000,0.987,0.975,0.522,0.048,1.000,...,0.000,0.000,0.000,0.000,0.0,0.048,0.048,0.048,0.048,0.048
301,01334,Saint-André-d'Huiriat,01,256,1.000,0.942,0.881,0.819,0.819,1.000,...,0.000,0.000,0.000,0.000,0.0,0.819,0.819,0.819,0.819,0.819
302,01335,Saint-André-le-Bouchoux,01,143,0.986,0.007,0.007,0.007,0.007,0.986,...,0.000,0.000,0.000,0.000,0.0,0.007,0.007,0.007,0.007,0.007
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36619,97419,Sainte-Rose,974,2723,0.999,0.960,0.838,0.396,0.000,0.999,...,0.000,0.000,0.000,0.000,0.0,0.000,0.000,0.000,0.000,0.000
36620,97420,Sainte-Suzanne,974,8901,0.995,0.958,0.936,0.776,0.722,0.995,...,0.000,0.000,0.000,0.000,0.0,0.722,0.722,0.722,0.722,0.722
36626,97502,Saint-Pierre,975,2653,0.991,0.991,0.991,0.991,0.000,0.000,...,0.991,0.991,0.991,0.991,0.0,0.000,0.000,0.000,0.000,0.000
36644,97701,Saint-Barthélemy,977,4693,1.000,0.936,0.762,0.146,0.000,1.000,...,0.000,0.000,0.000,0.000,0.0,0.000,0.000,0.000,0.000,0.000


#### Etudier les multiples options de read_csv

En particulier:

<pre>
pd.read_csv(
    <strong>filepath_or_buffer: Union[str, pathlib.Path, IO[~AnyStr]],</strong>
    <strong>sep=',',</strong>
    delimiter=None,
    <strong>header='infer',</strong>
    <strong>names=None,</strong>
    <strong>index_col=None,</strong>
    <strong>usecols=None,</strong>
    squeeze=False,
    prefix=None,
    mangle_dupe_cols=True,
    <strong>dtype=None,</strong>
    <strong>engine=None,</strong>
    <strong>converters=None,</strong>
    true_values=None,
    false_values=None,
    skipinitialspace=False,
    <strong>skiprows=None,</strong>
    <strong>skipfooter=0,</strong>
    nrows=None,
    <strong>na_values=None,</strong>
    <strong>keep_default_na=True,</strong>
    na_filter=True,
    verbose=False,
    skip_blank_lines=True,
    parse_dates=False,
    infer_datetime_format=False,
    keep_date_col=False,
    date_parser=None,
    dayfirst=False,
    cache_dates=True,
    iterator=False,
    <strong>chunksize=None,</strong>
    compression='infer',
    <strong>thousands=None,</strong>
    <strong>decimal: str = '.',</strong>
    lineterminator=None,
    quotechar='"',
    quoting=0,
    doublequote=True,
    escapechar=None,
    comment=None,
    encoding=None,
    dialect=None,
    error_bad_lines=True,
    warn_bad_lines=True,
    delim_whitespace=False,
    low_memory=True,
    memory_map=False,
    float_precision=None,
)
</pre>

In [ ]:
#pd.read_csv?

#### Analyse automatique avec pandas_profiling

**ATTENTION, il vaut mieux installer `pandas_profiling` dans un nouvel environnement**

<pre>
conda create --name profiling

activate profiling OU conda activate profiling

conda install -c conda-forge pandas-profiling
</pre>

In [ ]:
# profiling raw people
from pandas_profiling import ProfileReport

df = pd.read_csv('people.csv')

profile = ProfileReport(df, title='Pandas Profiling Report', explorative=True)

profile.to_file("people.html")

In [ ]:
# profiling clean people
profile = ProfileReport(clean_people(df), title='Pandas Profiling Report', explorative=True)

profile.to_file("clean_people.html")